In [5]:
# reports_origin.csv 에서 정제 안됐던 '연수국가 및 대학' 컬럼명 대학명(영문)으로 바꾼 뒤, 정제해서 school_info_refined.csv 파일의 대학명(영문) 컬럼과 최대한 매칭시키는 스크립트
# 이거 수행하고 다시 reports_summary_v2.py 실행하세요

import pandas as pd
import re
from datetime import datetime

# 데이터 로딩
reports_df = pd.read_csv("reports_merged.csv")
school_info_df = pd.read_csv("school_info_refined.csv")

# pandas 출력 옵션 설정 (생략 없이 전체 보기)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)

# rename, 문자열 공백 제거
reports_df = reports_df.rename(columns={'연수국가 및 대학': '대학명(영문)'})
reports_df['대학명(영문)'] = reports_df['대학명(영문)'].astype(str).str.strip()
school_info_df['대학명(영문)'] = school_info_df['대학명(영문)'].astype(str).str.strip()

# 원본 reports df
original_reports_df = reports_df.copy()

# 정제 함수
def clean_university_name(name):
    if isinstance(name, str):
        name = re.sub(r'\s+', ' ', name).strip()
        name = re.sub(r'^\s*(\[[^\]]*\])?\s*(\d{2}-\d(학기)?\s*)?\s*[가-힣]+\s+', '', name)
        return name.strip()
    return name

# 1: 왠지 모르겠는데 두 번 실행해야 제대로 정제됨.... 
reports_df['대학명_원본'] = reports_df['대학명(영문)']
reports_df['대학명(영문)'] = reports_df['대학명_원본'].apply(clean_university_name)
# 2: 왠지 모르겠는데 두 번 실행해야 제대로 정제됨.... 
reports_df['대학명_원본'] = reports_df['대학명(영문)']
reports_df['대학명(영문)'] = reports_df['대학명_원본'].apply(clean_university_name)
reports_df = reports_df.drop(columns=['대학명_원본'])



# # 확인
# print("508:", repr(reports_df.loc[508, '대학명(영문)']))
# print("740:", repr(reports_df.loc[740, '대학명(영문)']))

reports_df['대학명(영문)']
# 한자 포함된 행 필터링 및 덮어쓰기 수행
hanja_rows = reports_df['대학명(영문)'].str.contains(r'[\u4e00-\u9fff]', regex=True, na=False)

for idx, row in reports_df[hanja_rows].iterrows():
    hanja_name = row['대학명(영문)']
    matches = school_info_df[school_info_df['대학명(영문)'].str.contains(re.escape(hanja_name), na=False)]
    unique_matches = matches['대학명(영문)'].unique()
    
    # 모두 동일한 값이면 덮어쓰기
    if len(unique_matches) == 1 or (len(unique_matches) > 1 and len(set(unique_matches)) == 1):
        reports_df.at[idx, '대학명(영문)'] = unique_matches[0]

# reports_df 기준으로 school_info_df에 존재하지 않는 대학명 추출
unmatched_universities = reports_df[~reports_df['대학명(영문)'].isin(school_info_df['대학명(영문)'])]

# # 결과 출력
# print(f"school_info_df에 존재하지 않는 대학 수: {len(unmatched_universities.drop_duplicates(subset='대학명(영문)'))}")
# print(unmatched_universities[['대학명(영문)']].drop_duplicates())


# school_info_df에 존재하지 않는 대학명 가진 행 삭제
reports_df = reports_df[reports_df['대학명(영문)'].isin(school_info_df['대학명(영문)'])].reset_index(drop=True)

# 결과 저장 (선택 사항)
reports_df.to_csv("reports_cleaned.csv", index=False, encoding='utf-8-sig')

print(f"정제 후 reports_df 행 수: {len(reports_df)}")



정제 후 reports_df 행 수: 349


In [ ]:

# NaN 보존용 마스킹
nan_mask = reports_df['대학명(영문)'].isna()

# school 정보 리스트
school_names = school_info_df['대학명(영문)'].dropna().unique().tolist()

# 복사본 생성
updated_df = reports_df.copy()

# 1. reports_univ ⊂ school_univ
for idx, row in updated_df.iterrows():
    report_univ = row['대학명(영문)']
    if pd.isna(report_univ):  # NaN인 경우 스킵
        continue
    for school_univ in school_names:
        if report_univ in school_univ and report_univ != school_univ:
            updated_df.at[idx, '대학명(영문)'] = school_univ
            break

# 2. school_univ ⊂ reports_univ
for idx, row in updated_df.iterrows():
    report_univ = row['대학명(영문)']
    if pd.isna(report_univ):  # NaN인 경우 스킵
        continue
    for school_univ in school_names:
        if school_univ in report_univ and school_univ != report_univ:
            updated_df.at[idx, '대학명(영문)'] = school_univ
            break

# NaN 복원
updated_df.loc[nan_mask, '대학명(영문)'] = pd.NA

from fuzzywuzzy import fuzz
import pandas as pd

# 대학명 리스트
school_names = school_info_df['대학명(영문)'].dropna().unique().tolist()

# 복사본 생성
fuzzy_updated_df = updated_df.copy()

# 변경 수 카운트
changed_count = 0

# 덮어쓰기 수행 + 변경 로그 출력
for idx, row in fuzzy_updated_df.iterrows():
    report_univ = row['대학명(영문)']
    if pd.isna(report_univ):
        continue

    best_match = None
    best_score = 0

    for school_univ in school_names:
        score = fuzz.token_set_ratio(report_univ, school_univ)
        if score > best_score:
            best_match = school_univ
            best_score = score

    if best_score >= 97 and report_univ != best_match:
        # print(f"[{idx}] 유사도 {best_score}% | '{report_univ}' → '{best_match}'")
        fuzzy_updated_df.at[idx, '대학명(영문)'] = best_match
        changed_count += 1

# 결과 저장
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
output_path = f"reports_refined_{timestamp}.csv"
fuzzy_updated_df.to_csv(output_path, index=False, encoding='utf-8-sig')


# # 변경 수 출력
# print(f"\n유사도 97 이상으로 덮어쓴 총 행 수: {changed_count}")



# # '대학명(영문)' 값이 달라진 행 비교 (NaN도 포함해서 비교)
# changed_mask = original_reports_df['대학명(영문)'].fillna('') != fuzzy_updated_df['대학명(영문)'].fillna('')

# # 변경된 행만 추출 (컬럼 구조는 fuzzy_updated_df 기준으로 유지)
# changed_only_df = fuzzy_updated_df[changed_mask]

# # CSV로 저장
# timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
# output_path = f"reports_with_only_changed_{timestamp}.csv"
# changed_only_df.to_csv(output_path, index=False, encoding='utf-8-sig')

# # 변경된 행 개수 출력
# print(f"'대학명(영문)'이 변경된 행 수: {len(changed_only_df)}")



[6] 유사도 100% | 'Osaka University of Economics and Law (大阪経済法科大学)' → 'Osaka University of Economics and Law
(大阪経済法科大学)'
[76] 유사도 100% | 'College of Humanities & Sciences of Northeast Normal University (东北师范大学人文学院)' → 'College of Humanities & Sciences of 
Northeast Normal University
(东北师范大学人文学院)'
[118] 유사도 100% | 'Osaka University of Economics and Law (大阪経済法科大学)' → 'Osaka University of Economics and Law
(大阪経済法科大学)'
[120] 유사도 100% | 'College of Humanities & Sciences of Northeast Normal University (东北师范大学人文学院)' → 'College of Humanities & Sciences of 
Northeast Normal University
(东北师范大学人文学院)'
[203] 유사도 100% | 'Osaka University of Economics and Law (大阪経済法科大学)' → 'Osaka University of Economics and Law
(大阪経済法科大学)'
[205] 유사도 100% | 'Beijing University of Posts and Telecommunications (北京邮电大学)' → 'Beijing University of Posts and Telecommunications
(北京邮电大学)'
[209] 유사도 100% | 'College of Humanities & Sciences of Northeast Normal University (东北师范大学人文学院)' → 'College of Humanities & Sciences of 
North

In [ ]:
from fuzzywuzzy import fuzz
import pandas as pd

# 유사도 결과 저장
fuzzy_matches = []

# 기준 리스트
school_names = school_info_df['대학명(영문)'].dropna().unique().tolist()

# 각 report_univ에 대해 school_info_df에서 유사한 값 찾기
for report_univ in updated_df['대학명(영문)'].dropna().unique():
    for school_univ in school_names:
        similarity = fuzz.token_set_ratio(report_univ, school_univ)
        if similarity >= 80 and report_univ != school_univ:
            fuzzy_matches.append({
                'reports_univ': report_univ,
                'school_candidate': school_univ,
                'similarity_score': similarity
            })

# 결과 DataFrame
fuzzy_df = pd.DataFrame(fuzzy_matches)

# 유사도 기준 정렬
fuzzy_df = fuzzy_df.sort_values(by='similarity_score', ascending=False)

# 일부 출력
print(fuzzy_df.head(30))

# CSV 저장
fuzzy_df.to_csv("대학명_유사도_매칭후보.csv", index=False, encoding='utf-8-sig')
# 확인 결과 유사도 97% 이상은 덮어쓰기 해도 ㄱㅊ을듯


                                                    reports_univ  \
2817                        University of California Los Angeles   
1121                    California State University Monterey Bay   
3301                                       university of Ostrava   
2147                          Moscow State Linguistic University   
2800                              San Francisco state university   
3251                                     York St John university   
3287                               Brno University Of Technology   
2694  Hochschule Kaiserslautern (University of Applied Sciences)   
3309                                          algebra university   
2218                                       Tomas Bata university   
2131                        Worms University of applied sciences   
1628                                         American University   
2266                         International Chritstian University   
2671                        Saxion University of

In [ ]:
# # 1. 비교 DataFrame 생성
# comparison_df = pd.DataFrame({
#     '정제전_대학명(영문)': reports_df['대학명_원본'],
#     '정제후_대학명(영문)': updated_df['대학명(영문)'],
# })

# # 2. 정제 후 값이 school_info_df에 존재하면 그대로, 아니면 NaN
# school_names_set = set(school_info_df['대학명(영문)'].dropna().str.strip())

# comparison_df['school_info_매칭여부'] = comparison_df['정제후_대학명(영문)'].apply(
#     lambda x: x if x in school_names_set else pd.NA
# )

# # 3. CSV 저장
# comparison_df.to_csv("대학명_정제_비교결과.csv", index=False, encoding='utf-8-sig')


In [ ]:
# import pandas as pd

# # CSV 파일 불러오기
# df = pd.read_csv("대학명_정제_비교결과.csv")

# # NaN 개수 세기
# nan_count = df['school_info_매칭여부'].isna().sum()

# # 출력
# print(f'"school_info_매칭여부" 컬럼의 NaN 개수: {nan_count}')


"school_info_매칭여부" 컬럼의 NaN 개수: 435


In [ ]:
# # 기준 리스트
# school_names_set = set(school_info_df['대학명(영문)'].dropna().str.strip())

# # reports_df 기준으로 매칭 안 되는 값 필터링
# unmatched_universities = updated_df[~updated_df['대학명(영문)'].isin(school_names_set)]

# # 출력
# print(f"school_info_df에 존재하지 않는 대학 수: {unmatched_universities['대학명(영문)'].shape[0]}")
# print(f"school_info_df에 존재하지 않는 대학 수: {unmatched_universities['대학명(영문)'].nunique()}")
# print(unmatched_universities[['대학명(영문)']].drop_duplicates())


school_info_df에 존재하지 않는 대학 수: 435
school_info_df에 존재하지 않는 대학 수: 165
                                                                                      대학명(영문)
2                                                                The Florida State University
3                                                                   University of Mississippi
7                                                                Viadrina European University
8                                                    California State University, Bakersfield
9                                                                   Hanzehogeschool Groningen
10                                                                          Saxion Hogeschool
13                                                                 Europäische Fachhochschule
15                                                                           Banku Augstskola
17                                                                       Whitworth Uni

In [ ]:
# 결과 저장
updated_df.to_csv("reports_univ_normalized.csv", index=False, encoding='utf-8-sig')

In [ ]:

# # 리스트 추출
# reports_names = reports_df['대학명(영문)'].dropna().unique().tolist()
# school_names = school_info_df['대학명(영문)'].dropna().unique().tolist()

# # 1. reports_univ ⊂ school_univ (부분 포함, 단 일치 제외)
# subset_matches = []
# for report_univ in reports_names:
#     for school_univ in school_names:
#         if report_univ in school_univ and report_univ != school_univ:
#             subset_matches.append({
#                 'reports_univ': report_univ,
#                 'matched_school_univ': school_univ
#             })

# # 2. school_univ ⊂ reports_univ (반대 방향)
# superset_matches = []
# for school_univ in school_names:
#     for report_univ in reports_names:
#         if school_univ in report_univ and school_univ != report_univ:
#             superset_matches.append({
#                 'school_univ': school_univ,
#                 'matched_reports_univ': report_univ
#             })

# # DataFrame 변환 및 저장
# pd.DataFrame(subset_matches).to_csv("partial_subset_matches.csv", index=False, encoding='utf-8-sig')
# pd.DataFrame(superset_matches).to_csv("partial_superset_matches.csv", index=False, encoding='utf-8-sig')

In [ ]:
# # 1. 한자 문자열만 필터링
# hanja_reports = reports_df[reports_df['대학명(영문)'].str.contains(r'[\u4e00-\u9fff]', regex=True)]

# # 2. 결과 저장할 리스트
# results = []

# # 3. 각 한자 대학명이 school_info_df에 포함되는지 확인
# for name in hanja_reports['대학명(영문)'].unique():
#     matches = school_info_df[school_info_df['대학명(영문)'].str.contains(re.escape(name), na=False)]
#     results.append({
#         '한자 대학명': name,
#         '매칭된 school_info 대학 수': len(matches),
#         '매칭 대학명 리스트': matches['대학명(영문)'].tolist()
#     })

# # 4. 결과 출력
# for result in results:
#     print(f"🔎 한자 대학명: {result['한자 대학명']}")
#     print(f"   매칭된 개수: {result['매칭된 school_info 대학 수']}")
#     for match in result['매칭 대학명 리스트']:
#         print(f"     - {match}")
#     print('-' * 60)


🔎 한자 대학명: Soochow University(東吳大學)
   매칭된 개수: 1
     - Soochow University(東吳大學)
------------------------------------------------------------
🔎 한자 대학명: 明治学院大学
   매칭된 개수: 0
------------------------------------------------------------
🔎 한자 대학명: Sapporo Gakuin University(札幌学院大学)
   매칭된 개수: 1
     - Sapporo Gakuin University(札幌学院大学)
------------------------------------------------------------
🔎 한자 대학명: Shenzhen University(深圳大学)
   매칭된 개수: 1
     - Shenzhen University(深圳大学)
------------------------------------------------------------
🔎 한자 대학명: 西京学院
   매칭된 개수: 0
------------------------------------------------------------
🔎 한자 대학명: Meiji Gakuin University(明治学院大学)
   매칭된 개수: 0
------------------------------------------------------------
🔎 한자 대학명: Beijing University of Posts and Telecommunications
(北京邮电大学)
   매칭된 개수: 1
     - Beijing University of Posts and Telecommunications
(北京邮电大学)
------------------------------------------------------------
🔎 한자 대학명: Donghua University (东华大学)
   매칭된 개수: 2
 